## 找到動畫瘋如何表示集數
一部動畫的id會在參數中以`sn`表示  
以[https://ani.gamer.com.tw/animeVideo.php?sn=34142](https://ani.gamer.com.tw/animeVideo.php?sn=34142)為例  
`sn`是34142

In [ ]:
sn = '34142'

## 觀察network流量
先看到了`playlist.m3u8`，用ffplay播放

In [ ]:
! ffplay -i https://bahamut.akamaized.net/ad/welcome_to_anigamer/playlist.m3u8

但顯然這個播放清單是歡迎畫面的，不是動畫本體

觀察有可能的API
- `getdeviceid.php`
- `video.php`
- `token.php`

## 初始化工作階段
後續會呼叫的API有一部分會認cookie，且都會認header，這些在開發的時候會一個一個測試，但是這邊就先寫結論方便展示  
header會認以下幾個：
- user-agent
- referer
- origin

In [ ]:
import requests

header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70', 'referer': 'https://ani.gamer.com.tw/animeVideo.php', 'origin': 'https://ani.gamer.com.tw'}
session = requests.session()
session.headers.update(header)
print(session.cookies)

### 載入現有的cookie

如果有某帳號的關鍵cookie的話，就可以用那個帳號的權限來瀏覽動畫，例如有帳號就能看720p，有會員就能看1080p

In [ ]:
with open('cookies.txt', 'r') as f:
    cookies = f.read().strip()
    cookies = {i.split('=')[0]: i.split('=')[1] for i in cookies.split('\n')}
    session.cookies.update(cookies)

## 觀看廣告
沒有買會員的話就需要看30秒廣告才能開始看動畫，但這個30秒怎麼計數的呢？  
開始廣告之後會發現一個API呼叫
- `videoCastcishu.php?s=...&sn=37780`

廣告結束後也有一個呼叫
- `videoCastcishu.php?s=...&sn=37780&ad=end`

如果是客戶端計數，直接無視就好，如果是伺服器端計數，就乖乖等

### 取得廣告的ID
生成廣告ID是直接從原始碼幹出來用python復刻的，可以當作魔法

In [ ]:
import functions
ad_data = functions.get_major_ad()
session.cookies.update(ad_data['cookie'])
print(ad_data)

### 模擬看廣告
將廣告ID送出去，假裝自己開始看了廣告，計時之後再送看完了的封包  
廣告原本是30秒，但實際上伺服器端計時約25秒

In [ ]:
import time

print('start ad')
session.get(f'https://ani.gamer.com.tw/ajax/videoCastcishu.php?s={ad_data["adsid"]}&sn={sn}')
ad_countdown = 25
for countdown in range(ad_countdown):
    print(f'ad {ad_countdown - countdown}s remaining', end='\r')
    time.sleep(1)

session.get(f'https://ani.gamer.com.tw/ajax/videoCastcishu.php?s={ad_data["adsid"]}&sn={sn}&ad=end')
print('\nend ad')

In [ ]:
print(session.cookies)

## 取得播放清單
可以觀察到一個叫做`m3u8.php`的API裡面取得了m3u8的網址，例如下面這樣
- https://ani.gamer.com.tw/ajax/m3u8.php?sn=37780&device=0052f38c1634575e5e81f13b1daef008c6ae921bc6415a7d660cea1c4571

request參數裡面包含了deviceid，經過測試之後這是必要的參數  
而`m3u8.php`傳回內容是一個m3u8的網址

### 取得devideid

In [ ]:
import json

deviceid_res = session.get('https://ani.gamer.com.tw/ajax/getdeviceid.php')
deviceid_res.raise_for_status()
deviceid = json.loads(deviceid_res.text)['deviceid']
print(deviceid)

### 取得播放清單網址

In [ ]:
m3u8_php_res = session.get(f'https://ani.gamer.com.tw/ajax/m3u8.php?sn={sn}&device={deviceid}')
m3u8_php_res.raise_for_status()
try:
    playlist_basic_url = json.loads(m3u8_php_res.text)['src']
except Exception as ex:
    print(m3u8_php_res.text)
    print('failed to load m3u')
print(playlist_basic_url)

### 取得第一層播放清單
經由剝參數測試，這個資源會檢查header  
> 為何已經取得了m3u8卻不直接用ffmpeg下載？  

因為檢查的header有User-Agent、referer、origin，給ffmpeg時需要用`-header`參數傳遞，每個header一行，但是在windows環境下沒辦法傳遞換行字元至參數中，所以選擇自己寫下載的部分，在本機合併

In [ ]:
import os

meta_base = os.path.dirname(playlist_basic_url) + '/'
playlist_basic_res = requests.get(playlist_basic_url, headers=header)
playlist_basic_res.raise_for_status()

print(playlist_basic_res.text)

### 分離出第二層播放清單網址
第一層播放清單中引用了第二層播放清單，未登入帳號只能看到360p的播放清單  
題外話，未登入一直以來都只能看360p，但以前其實未登入播放清單會給到720p，在前端動點手腳解除UI封印就可以播放了

In [ ]:
playlist_basic = playlist_basic_res.text.split('\n')
resolutions_metadata = []
for i in range(len(playlist_basic)):
    line = playlist_basic[i]
    if line.startswith('#EXT-X-STREAM-INF'):
        resolutions_metadata.append({'info': line[18:], 'url': playlist_basic[i+1]})

In [ ]:
print(json.dumps(resolutions_metadata, indent=4))

### 列出可選的畫質
未登入的工作階段只能看到360p

In [ ]:
for j in range(len(resolutions_metadata)):
    print('#%s: %s' % (j, resolutions_metadata[j]['info']))

In [ ]:
selected_resolution = int(input('select a resolution: '))
print(f'selected: #{selected_resolution}')

## 準備下載

### 準備下載用的資料夾

In [ ]:
resolution_number = resolutions_metadata[selected_resolution]['info'].rsplit('=', 1)[1]
folder_name = f'{sn}_{resolution_number}'
os.makedirs(os.path.join('Download', folder_name), exist_ok=True)
os.chdir(os.path.join('Download', folder_name))

### 取得第二層播放清單
經由剝參數測試，這個資源也會檢查header

In [ ]:
chunklist_url = meta_base + resolutions_metadata[selected_resolution]['url']
print(chunklist_url)

In [ ]:
chunklist_res = requests.get(chunklist_url, headers=header)
print(chunklist_res.text)

### 儲存m3u8至檔案

In [ ]:
chunklist_filename = folder_name + '.m3u8'
with open(chunklist_filename, 'wb') as chunklist_file:
    for chunk in chunklist_res:
        chunklist_file.write(chunk)

### 分離出影片網址的base

In [ ]:
chunks_base = meta_base + resolutions_metadata[selected_resolution]['url'].rsplit('/', 1)[0] + '/'
print(chunks_base)

#### 初始化多執行緒下載模組
自己手刻的土炮多執行緒，但記得隊網站斯文一些才有禮貌，不要一傢伙榨乾對方頻寬

In [ ]:
import multiple_thread_downloading
chunklist = chunklist_res.text.split('\n')
mtd_worker = multiple_thread_downloading.mtd(header, chunks_base, chunklist_res.text.count('.ts'))

#### 把.ts網址丟進queue
裡面有key的檔案，也順便丟進去下載

In [ ]:
import re

for k in range(len(chunklist)):
    line = chunklist[k]
    if line.startswith('#EXTINF'):
        ts_name = chunklist[k+1]
        # push
        mtd_worker.push(ts_name)
    elif line.startswith('#EXT-X-KEY'):
        key_name = re.match('.*URI="(.*)".*$', line).group(1)
        # push
        mtd_worker.push(key_name)

### 等待下載完成

In [ ]:
mtd_worker.join()

## 呼叫ffmpeg合併
key在m3u8裡面有寫到，ffmpeg會自己解開加密的部分

In [ ]:
command = f'ffmpeg -allowed_extensions ALL -i {chunklist_filename} -c copy {folder_name}.mp4'
print(command)
os.system(command)